In [2]:
%pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [3]:
input_directory = 'D:\\PhD-data\\denoising\\pre'
output_directory = 'D:\\PhD-data\\denoising\\post'

In [5]:
import os
import librosa
import numpy as np
import soundfile as sf

# Function to apply spectral subtraction to an audio frame
def apply_spectral_subtraction(frame, noise_spectrum):
    # Apply Fourier transform
    spectrum = np.fft.fft(frame)
    
    # Perform spectral subtraction
    denoised_spectrum = np.maximum(np.abs(spectrum) - noise_spectrum, 0)
    
    # Apply inverse Fourier transform
    denoised_frame = np.fft.ifft(denoised_spectrum)
    
    return denoised_frame.real  # Convert to real values

# Parameters
frame_duration = 4.5  # Duration of each frame in seconds

# Iterate over the directory structure
for root, dirs, files in os.walk(input_directory):
    # Create corresponding directories in the output structure
    output_root = root.replace(input_directory, output_directory)
    os.makedirs(output_root, exist_ok=True)
    
    # Iterate over the audio files
    for file in files:
        if file.endswith('.wav'):
            # Load the audio file
            audio_path = os.path.join(root, file)
            audio, sr = librosa.load(audio_path, sr=None)
            
            # Calculate the number of frames
            frame_length = int(frame_duration * sr)
            num_frames = len(audio) // frame_length
            
            # Apply spectral subtraction to each frame
            denoised_audio = []
            for i in range(num_frames):
                start = i * frame_length
                end = (i + 1) * frame_length
                
                # Extract frame
                frame = audio[start:end]
                
                # Estimate noise spectrum (assuming it is stationary)
                if i == 0:
                    noise_spectrum = np.mean(np.abs(np.fft.fft(frame)))
                
                # Apply spectral subtraction to the frame
                denoised_frame = apply_spectral_subtraction(frame, noise_spectrum)
                
                # Collect the denoised frames
                denoised_audio.extend(denoised_frame)
            
            # Save the denoised audio file
            output_path = os.path.join(output_root, file)
            sf.write(output_path, np.asarray(denoised_audio), sr)
